In [1]:
from Tirto import Tirto #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [3]:
# error 2018-04-27 halaman 2 
# start_date = date(2018, 5, 18) halaman 4
obj = Tirto()
end_date = date(2018, 9, 1)
start_date = date(2018, 8, 11)
init_page = 3

In [ ]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], init_page, single_date.strftime("%Y-%m-%d"))
    init_page = 1

2018-08-12
page  1
https://tirto.id/indeks/1?date=2018-08-12
https://tirto.id/demokrat-pemasangan-iklan-baliho-ahy-inisiatif-dari-kader-cR3d
Insert berita  Demokrat: Pemasangan Iklan Baliho AHY Inisiatif dari Kader
masuk
https://tirto.id/hasil-arsenal-vs-manchester-city-liga-inggris-2018-skor-akhir-0-2-cR3h
Insert berita  Hasil Arsenal vs Manchester City Liga Inggris 2018, Skor Akhir 0-2
masuk
https://tirto.id/bnpb-606-sekolah-rusak-akibat-gempa-lombok-termasuk-3051-kelas-cR2T
Insert berita  BNPB: 606 Sekolah Rusak Akibat Gempa Lombok, Termasuk 3.051 Kelas
masuk
https://tirto.id/kpu-diminta-sediakan-informasi-lengkap-profil-bakal-caleg-di-dcs-cR2K
Insert berita  KPU Diminta Sediakan Informasi Lengkap Profil Bakal Caleg di DCS
masuk
https://tirto.id/hasil-arsenal-vs-manchester-city-di-liga-inggris-babak-i-skor-0-1-cR2C
Insert berita  Hasil Arsenal vs Manchester City di Liga Inggris, Babak I Skor 0-1
masuk
https://tirto.id/jokowi-klaim-wapres-jk-mau-bergabung-di-tim-kampanyenya-cR2B
Inse